In [4]:
import os
import csv
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import train_test_split

In [5]:
traindata = pd.read_csv('/content/drive/MyDrive/train_bodies.csv')

testdata = pd.read_csv('/content/drive/MyDrive/competition_test_bodies.csv')

# load training data
traindf = pd.read_csv('/content/drive/MyDrive/train_stances.csv')
# load testing data
testdf = pd.read_csv('/content/drive/MyDrive/competition_test_stances.csv')

traindf.replace('agree',0,True)
traindf.replace('disagree',1,True)
traindf.replace('discuss',2,True)
traindf.replace('unrelated',3,True)

testdf.replace('agree',0,True)
testdf.replace('disagree',1,True)
testdf.replace('discuss',2,True)
testdf.replace('unrelated',3,True)

train_df_combined = traindf.join(traindata.set_index('Body ID'), on='Body ID')
test_df_combined = testdf.join(testdata.set_index('Body ID'), on='Body ID')

headlines = list()

for i in train_df_combined['Headline']:
    if i not in headlines:
        headlines.append(i)

temp_2 = train_df_combined.loc[train_df_combined['Stance'] == 3]

df = pd.DataFrame()
count, sum_1 = 0, 0

for i in headlines:
    temp_1 = train_df_combined.loc[train_df_combined['Headline'] == i]
    related_word = temp_1.loc[(temp_1['Stance'] == 0) | (temp_1['Stance'] == 1) | (temp_1['Stance'] == 2)]
    df_temp = df.append(related_word)
    df = df_temp
    unrelated_word = temp_1.loc[temp_1['Stance'] == 3]
    if related_word.shape[0] < 1:
        count = count + 1
        sum_1 = sum_1 + unrelated_word.shape[0]
        continue
    if unrelated_word.shape[0] < related_word.shape[0]:
        df_temp = df.append(unrelated_word)
    else:
        df_temp = df.append(unrelated_word.iloc[:int(related_word.shape[0])])
    df = df_temp
  
#df_temp = df.append(df1)
#df = df_temp.append(df2.iloc[:df1.shape[0]])

temp_2 = df[df['Stance'] == 3]

stances = df['Stance']
stances.replace(0, 'agree',True)
stances.replace(1, 'disagree',True)
stances.replace(2, 'discuss',True)
stances.replace(3, 'unrelated',True)

df['Stance'] = stances

df = df.sample(frac = 1)

df.to_csv('full_train.csv', index=False, encoding='utf-8')

stances_1 = test_df_combined['Stance']
stances_1.replace(0, 'agree',True)
stances_1.replace(1, 'disagree',True)
stances_1.replace(2, 'discuss',True)
stances_1.replace(3, 'unrelated',True)
test_df_combined['Stance'] = stances_1

test_df_combined.to_csv('full_test.csv', index=False, encoding='utf-8')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 5.1 MB/s 
     |████████████████████████████████| 1.8 MB 60.1 MB/s 
     |████████████████████████████████| 9.1 MB 47.1 MB/s 
     |████████████████████████████████| 4.7 MB 34.4 MB/s 
     |████████████████████████████████| 6.6 MB 48.7 MB/s 
     |████████████████████████████████| 365 kB 69.7 MB/s 
     |████████████████████████████████| 1.3 MB 66.4 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 596 kB 61.2 MB/s 
     |████████████████████████████████| 101 kB 13.6 MB/s 
     |████████████████████████████████| 181 kB 62.1 MB/s 
     |████████████████████████████████| 157 kB 62.0 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 157 kB 64.2 MB/s 
     |████████████████████████████████| 156 kB 66.2 MB/s 
     |██████████████████████████████

In [7]:
# https://stackoverflow.com/questions/56850711/importerror-please-install-apex-from-https-www-github-com-nvidia-apex-to-use

import os, sys, shutil
import time
import gc
from contextlib import contextmanager
from pathlib import Path
import random
import numpy as np, pandas as pd
from tqdm import tqdm, tqdm_notebook

@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')

USE_APEX = True

"""if USE_APEX:
            with timer('install Nvidia apex'):
                # Installing Nvidia Apex
                os.system('git clone https://github.com/NVIDIA/apex; cd apex; pip install -v --no-cache-dir' + 
                          ' --global-option="--cpp_ext" --global-option="--cuda_ext" ./')
                os.system('rm -rf apex/.git') # too many files, Kaggle fails
                from apex import amp"""

'if USE_APEX:\n            with timer(\'install Nvidia apex\'):\n                # Installing Nvidia Apex\n                os.system(\'git clone https://github.com/NVIDIA/apex; cd apex; pip install -v --no-cache-dir\' + \n                          \' --global-option="--cpp_ext" --global-option="--cuda_ext" ./\')\n                os.system(\'rm -rf apex/.git\') # too many files, Kaggle fails\n                from apex import amp'

In [8]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [9]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 10085, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 10085 (delta 98), reused 131 (delta 59), pack-reused 9884
Receiving objects: 100% (10085/10085), 14.96 MiB | 23.10 MiB/s, done.
Resolving deltas: 100% (6906/6906), done.
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/pytho

In [10]:
from simpletransformers.classification import ClassificationModel

In [11]:
import os, sys, shutil
import time
import gc
from contextlib import contextmanager
from pathlib import Path
import random
import numpy as np, pandas as pd
from tqdm import tqdm, tqdm_notebook

In [12]:
import pandas as pd
import numpy as np

In [13]:
trainraw = pd.read_csv('/content/drive/MyDrive/full_train.csv')
testraw = pd.read_csv('/content/drive/MyDrive/full_test.csv')

In [14]:
train = pd.DataFrame(columns=['text_a', 'text_b', 'labels'])
train['text_a'] = trainraw['Headline']
train['text_b'] = trainraw['articleBody']

In [15]:
stances = trainraw['Stance']
stances.replace('agree', 0,True)
stances.replace('disagree', 1, True)
stances.replace('discuss', 2, True)
stances.replace('unrelated', 3, True)
stances
train['labels'] = stances

In [16]:
stances = testraw['Stance']
stances.replace('agree', 0,True)
stances.replace('disagree', 1, True)
stances.replace('discuss', 2, True)
stances.replace('unrelated', 3, True)
stances

0        3
1        3
2        3
3        3
4        3
        ..
25408    0
25409    2
25410    1
25411    1
25412    0
Name: Stance, Length: 25413, dtype: int64

In [17]:
test = pd.DataFrame(columns=['text_a', 'text_b'])

test['text_a'] = testraw['Headline']
test['text_b'] = testraw['articleBody']

In [18]:
test['labels'] = stances

In [19]:
model_final = ClassificationModel('roberta', 'roberta-base',  num_labels=4, args={
    'learning_rate':1e-5,
    'num_train_epochs': 1,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'process_count': 10,
    'evaluate_during_training_steps': False,
    'train_batch_size': 4,
    'eval_batch_size': 4,
    'max_seq_length': 512,
    'fp16': True,
    'early_stopping': True
})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [20]:
model_final.train_model(train)

  0%|          | 0/26854 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/6714 [00:00<?, ?it/s]

(6714, 0.3981670357968291)

In [21]:
test_ll = list()

for i in range(len(test)):
  test_ll.append([test['text_a'][i], test['text_b'][i]])

In [22]:
def accuracy(pred):
  count = 0
  for i in range(len(pred)):
    if pred[i] == stances[i]:
      count = count + 1
  print(count / len(pred))

In [23]:
def createAnswer(opt, epoch):
  dfanswer = pd.DataFrame(columns=['Headline', 'Body ID', 'Stance'])
  dfanswer['Stance'] = opt
  dfanswer.replace(0, 'agree',True)
  dfanswer.replace(1, 'disagree',True)
  dfanswer.replace(2, 'discuss',True)
  dfanswer.replace(3, 'unrelated',True)
  dfanswer['Headline'] = testraw['Headline']
  dfanswer['Body ID'] = testraw['Body ID']
  fName = 'answerEp' + str(epoch) + '.csv'
  dfanswer.to_csv(fName, index=False, encoding='utf-8')
  return dfanswer

In [24]:
model_final = ClassificationModel(
    "roberta", "outputs/checkpoint-6714-epoch-1"
)

In [25]:
pred, raw_outputs = model_final.predict(test_ll)

  0%|          | 0/25413 [00:00<?, ?it/s]

  0%|          | 0/6354 [00:00<?, ?it/s]

In [26]:
accuracy(pred)

0.9310589068586944


In [27]:
createAnswer(pred, 5)

,Headline,Body ID,Stance
0,Ferguson riots: Pregnant woman loses eye after...,2008,unrelated
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,1550,unrelated
2,A Russian Guy Says His Justin Bieber Ringtone ...,2,unrelated
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",1793,unrelated
4,Argentina's President Adopts Boy to End Werewo...,37,unrelated
...,...,...,...
25408,The success of the Affordable Care Act is a hu...,2582,agree
25409,The success of the Affordable Care Act is a hu...,2583,agree
25410,The success of the Affordable Care Act is a hu...,2584,discuss
25411,The success of the Affordable Care Act is a hu...,2585,agree


In [28]:
dfanswer = pd.DataFrame(columns=['Headline', 'Body ID', 'Stance'])

In [29]:
dfanswer['Stance'] = pred

In [30]:
dfanswer.replace(0, 'agree',True)
dfanswer.replace(1, 'disagree',True)
dfanswer.replace(2, 'discuss',True)
dfanswer.replace(3, 'unrelated',True)

In [31]:
dfanswer['Headline'] = testraw['Headline']
dfanswer['Body ID'] = testraw['Body ID']

In [32]:
dfanswer.to_csv('answer.csv', index=False, encoding='utf-8')